In [1]:
import pickle

def pickle_load(n):
    with open('./features/Model2_no_stopwords/fiction_' + str(n) + '.pickle', 'rb') as f:
        fic = pickle.load(f)

    with open('./features/Model2_no_stopwords/non_fiction_' + str(n) + '.pickle', 'rb') as f:
        nonfic = pickle.load(f)

    return fic, nonfic

fic500, nonfic500   = pickle_load(500)

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.linear_model import LogisticRegression


'''
For 500:
'''
# Getting it ready for Count Vectorizer:
countvec_fiction500 = []
countvec_nonfiction500 = []

for doc in fic500:
    temp = ' '.join(doc)
    countvec_fiction500.append(temp)
    
for doc in nonfic500:
    temp = ' '.join(doc)
    countvec_nonfiction500.append(temp)
    
fiction_plus_nonfiction500 = countvec_fiction500 + countvec_nonfiction500
    
vectorizer500 = CountVectorizer()
X500 = vectorizer500.fit_transform(fiction_plus_nonfiction500)

In [3]:
print X500.shape
labels = [1]*4558 + [0]*4558
print len(labels)
# Train LogReg Model 2 for 500 words:
model = LogisticRegression(C=1000, penalty='l2', solver='lbfgs')
model.fit(X500, labels)

(9116, 154749)
9116


LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

In [5]:
'''
Calculating CountVec feature vectors for test-fiction-big.
'''

with open('./features/test_big_fiction_500_Model2.pickle', 'rb') as f:
    features_fiction_big = pickle.load(f)

# Getting it ready for Count Vectorizer:
countvec_500 = []

for doc in features_fiction_big:
    temp = ' '.join(doc)
    countvec_500.append(temp)
    
Xtest_500 = vectorizer500.transform(countvec_500)

print Xtest_500.shape

(15036, 154749)


In [6]:
# Now, predicting:
predictions = model.predict(Xtest_500)

# Calculating accuracy (assuming all is fiction in Fiction_big)
from sklearn.metrics import accuracy_score

true = [1]*15036
print accuracy_score(true, predictions)

0.528198989093


In [1]:
# Getting the order:
import pickle

# Code to pickle this in Pickle-FictionBig-Model2 notebook:
with open('./order_fiction.pickle', 'rb') as f:
    order_fiction = pickle.load(f)
print order_fiction

In [8]:
print len(order_fiction)
print len(predictions)
main = zip(order_fiction, predictions)
print main[:4]

15036
15036
[(('0000200100.xml.txt', 'LitAndLang_1'), 1), (('0000200200.xml.txt', 'LitAndLang_1'), 1), (('0000200400.xml.txt', 'LitAndLang_1'), 1), (('0000200500.xml.txt', 'LitAndLang_1'), 1)]


In [2]:
final_fiction = []

# main has filename + its prediction
for tup, p in main:
    if p == 1:
        final_fiction.append(tup)
        
print len(final_fiction)
# print final_fiction

7942


In [3]:
finalfiction_fnames = list(zip(*final_fiction)[0])
print len(finalfiction_fnames)
# print finalfiction_fnames

7942


In [4]:
'''
Writing these final fiction filenames to the FINAL OUTPUT dataframe.
'''
import pandas as pd

final_fic_fnames = [f[:-4] for f in finalfiction_fnames]

fiction_big = pd.read_csv('./Fiction_Big.csv')

FinalFiction = fiction_big.loc[fiction_big['Filename'].isin(final_fic_fnames)]

In [42]:
# FinalFiction has 7942 rows (that were predicted by model). Adding the remaining 665 that were not present in 
# fiction_big but were present in fiction_small. For detailed analysis of that, see the last three cells of this 
# notebook

In [43]:
fiction_small = pd.read_csv('./Fiction_Small.csv')
fnames_fiction_small = fiction_small['Filename'].tolist()

count = 0
notinfinal_butinsmall = []
for fn in fnames_fiction_small:
    if fn not in final_fic_fnames:
        count += 1
        notinfinal_butinsmall.append(fn)
print count
print notinfinal_butinsmall[:8]

665
['0003900900.xml', '0004700401.xml', '0004700402.xml', '0004700403.xml', '0007901400.xml', '0076201200.xml', '0079600800.xml', '0086000501.xml']


In [5]:
NotInFinal_ButInSmall = fiction_small.loc[fiction_small['Filename'].isin(notinfinal_butinsmall)]

In [6]:
# Concatenating the predicted in fiction_big and NotInFinalButInSmall
FINAL_OUTPUT = pd.concat([FinalFiction, NotInFinal_ButInSmall])

In [47]:
print 7942+665
print FINAL_OUTPUT.nunique()

8607
Filename          8607
DocumentID        8607
ESTC_ID           5685
Date               105
Title             5268
Vol_Number          25
Author            1530
Imprint           5356
Field_Headings    2033
dtype: int64


In [57]:
# Writing to csv
FINAL_OUTPUT.to_csv('./FINAL_FICTION_LogReg_Model2.csv', index=None)

In [12]:
# Last check to make sure all fiction_small is in FinalOutput
import pandas as pd
fsmall = pd.read_csv('./Fiction_Small.csv')['Filename'].tolist()
output = pd.read_csv('./FINAL_FICTION_LogReg_Model2.csv')['Filename'].tolist()
print set(fsmall).issubset(set(output))

True


#### Checking which ECCO-Categories do these fiction files belong to:

In [23]:
output[:4]

['0000200100.xml', '0000200200.xml', '0000200400.xml', '0000200500.xml']

In [24]:
main_table = pd.read_csv('/Users/sunyambagga/Desktop/ECCO Paper Writing/UPDATED_Table1_Combined_and_Annotated.csv')
main_table.head()

,Filename,DocumentID,ESTC_ID,Date,Title,Vol_Number,Author,Imprint,Field_Headings,fn_no,nfn_no,ECCO_Category,ECCO_Collection,Predicted_Class
0,0031300100.xml,31300100,T021625,1736,"Bibliotheca topographica Anglicana: or, a new ...",0,"Worrall, John",London : printed for J. Worrall at the Dove in...,"Books, Prices, Catalogs, Booksellers', Great B...",0,63,Manifest_GenRef,ECCO_1,GenRef
1,0031300200.xml,31300200,T013049,1787,"A catalogue of books printed for, and sold by ...",0,"Dilly, Charles","[London], s.n, 1787.","Catalogs, Booksellers', Early works to 1800",0,32,Manifest_GenRef,ECCO_1,GenRef
2,0031300300.xml,31300300,T057382,1800,"Rules of a reading-society, established April ...",0,Anon,"London : printed by H.D. Steel, No. 51, Lothbu...","[London Reading Society], Rules and practice, ...",0,7,Manifest_GenRef,ECCO_1,GenRef
3,0031300400.xml,31300400,T012488,1787,Rules for regulating the subscription library ...,0,Anon,"Stamford : printed by Newcomb and Peat, [1787].","[Stamford Subscription Library], Rules and pra...",0,17,Manifest_GenRef,ECCO_1,GenRef
4,0031300500.xml,31300500,W029739,1773,"A catalogue of books, imported and to be sold ...",0,"Knox, Henry","[Boston : Sold by Henry Knox, 1773].","Booksellers and bookselling, Massachusetts, Bo...",0,39,Manifest_GenRef,ECCO_1,GenRef


In [25]:
fic_table = main_table.loc[main_table['Filename'].isin(output)]
fic_table.head()

,Filename,DocumentID,ESTC_ID,Date,Title,Vol_Number,Author,Imprint,Field_Headings,fn_no,nfn_no,ECCO_Category,ECCO_Collection,Predicted_Class
4353,0003900900.xml,3900900,T110586,1753,Memoires secrets pour servir <c3><a0> l'histoi...,0,"M. M. C. F., Ecu<c3><a8>ier","London : printed for R. Jennys, [1753?].","French fiction, 18th century",0,30,Manifest_HistAndGeo,ECCO_1,Fiction
4411,0004700401.xml,4700401,T125094,1748,Anecdotes de la cour de Fran<c3><a7>ois I. Par...,Volume 1,"Lussan, Marguerite de","Londres [i.e. Paris?] : chez Jean Nours [sic],...","France, Court and courtiers, France, History, ...",0,304,Manifest_HistAndGeo,ECCO_1,Fiction
4412,0004700402.xml,4700402,T125094,1748,Anecdotes de la cour de Fran<c3><a7>ois I. Par...,Volume 2,"Lussan, Marguerite de","Londres [i.e. Paris?] : chez Jean Nours [sic],...","France, Court and courtiers, France, History, ...",1,343,Manifest_HistAndGeo,ECCO_1,Fiction
4413,0004700403.xml,4700403,T125094,1748,Anecdotes de la cour de Fran<c3><a7>ois I. Par...,Volume 3,"Lussan, Marguerite de","Londres [i.e. Paris?] : chez Jean Nours [sic],...","France, Court and courtiers, France, History, ...",0,309,Manifest_HistAndGeo,ECCO_1,Fiction
4510,0007901400.xml,7901400,T069692,1782,"The history of the civil wars in Germany, from...",0,"Defoe, Daniel","Newark : printed by James Tomlinson, for the p...","Thirty Years' War, 1618-1648, Fiction, Great B...",2,379,Manifest_HistAndGeo,ECCO_1,Fiction


In [30]:
fic_table['ECCO_Category'].value_counts()

Manifest_LitAndLang      7954
Manifest_HistAndGeo       444
Manifest_SSAndFineArt     100
Manifest_RelandPhil        96
Manifest_MedSciTech        11
Manifest_Law                2
Name: ECCO_Category, dtype: int64

In [31]:
7954+444+100+96+11+2

8607

In [21]:
'''
Sanity check: Investigating if all of fiction_small is present in FinalFiction.
'''

import pandas as pd
fnames_fiction_small = pd.read_csv('./Fiction_Small.csv')['Filename'].tolist()

txt_fnames_fiction_small = [f + '.txt' for f in fnames_fiction_small]

count = 0
notinfinal_butinsmall = []
for fn in txt_fnames_fiction_small:
    if fn not in finalfiction_fnames:
        count += 1
        notinfinal_butinsmall.append(fn)

print "Files that are in fiction_small but not in final-fiction-fnames: ", count

Files that are in fiction_small but not in final-fiction-fnames:  665


In [24]:
# That is interesting, because there were exactly 665 files that were not in fiction_big. 
# If these are the same, that means the classifier predicted all fiction_small accurately.

small_and_big_intersection = fiction_big.loc[fiction_big['Filename'].isin(fnames_fiction_small)]
intersection_fnames = small_and_big_intersection['Filename'].tolist()
print "Fiction small: ", len(fnames_fiction_small)
print "Intersection: ", len(intersection_fnames)

notInBig = set(fnames_fiction_small) - set(intersection_fnames)
print "Not in fiction_big: ", len(notInBig)

Fiction small:  4558
Intersection:  3893
Not in fiction_big:  665


In [26]:
# Confirming if they are the same:
txt_notInBig = [s + '.txt' for s in notInBig]
set(txt_notInBig) == set(notinfinal_butinsmall)

True